In [1]:
import tools
import importlib
import memory
importlib.reload(tools)
importlib.reload(memory)

from tools import network_tools
from tools import firewall_tools

In [4]:
alerts = await network_tools.get_fast_alerts(time_window=2)
alerts

{'security_events': {'alerts': [{'message': '[**] [1:2010937:3] ET SCAN Suspicious inbound to mySQL port 3306 [**] [Classification: Potentially Bad Traffic] [Priority: 2] {TCP} 192.168.100.2:62023 -> 172.20.0.2:3306',
    'timestamp': '2025-08-06T10:10:48.339643+02:00'},
   {'message': '[**] [1:3400001:1] POSSBL PORT SCAN (NMAP -sS) [**] [Classification: Attempted Information Leak] [Priority: 2] {TCP} 192.168.100.2:62023 -> 172.20.0.2:143',
    'timestamp': '2025-08-06T10:10:48.439874+02:00'},
   {'message': '[**] [1:3400001:1] POSSBL PORT SCAN (NMAP -sS) [**] [Classification: Attempted Information Leak] [Priority: 2] {TCP} 192.168.100.2:62023 -> 172.20.0.2:161',
    'timestamp': '2025-08-06T10:10:48.442229+02:00'},
   {'message': '[**] [1:2010935:3] ET SCAN Suspicious inbound to MSSQL port 1433 [**] [Classification: Potentially Bad Traffic] [Priority: 2] {TCP} 192.168.100.2:62023 -> 172.20.0.2:1433',
    'timestamp': '2025-08-06T10:10:48.442253+02:00'},
   {'message': '[**] [1:2200003

In [20]:
network_tools.get_docker_containers()

{'honeypot_config': [{'id': '640353ced621',
   'name': 'cve-2014-6271-web-1',
   'image': 'vulhub/bash:4.3.0-with-httpd',
   'status': 'running',
   'created': '2025-08-06T07:48:02.488925808Z',
   'ports': ['80/tcp'],
   'ip_address': '172.20.0.7'},
  {'id': '1ba281ac4041',
   'name': 'cve-2015-5254-activemq-1',
   'image': 'vulhub/activemq:5.11.1',
   'status': 'running',
   'created': '2025-08-06T07:48:02.181490751Z',
   'ports': ['61616/tcp', '8161/tcp'],
   'ip_address': '172.20.0.6'},
  {'id': 'cab62e2456dd',
   'name': 'cve-2021-22205-gitlab-1-proxy',
   'image': 'nginx:alpine',
   'status': 'running',
   'created': '2025-08-06T07:48:01.190924283Z',
   'ports': ['80/tcp'],
   'ip_address': '172.20.0.10'},
  {'id': 'ce9f3c67fb64',
   'name': 's2-057-struts2-1',
   'image': 'vulhub/struts2:2.3.34-showcase',
   'status': 'running',
   'created': '2025-08-06T07:48:00.787196402Z',
   'ports': ['8080/tcp'],
   'ip_address': '172.20.0.2'}]}

In [22]:
print(network_tools.check_services_health())
rules = await firewall_tools.get_firewall_rules()
print(rules['firewall_config']['data']['rules'])

{'firewall_status': 'up', 'suricata_status': 'up'}
Chain FORWARD (policy DROP)
num  target     prot opt source               destination         
1    ACCEPT     all  --  0.0.0.0/0            0.0.0.0/0            state RELATED,ESTABLISHED
2    ACCEPT     icmp --  0.0.0.0/0            0.0.0.0/0           
3    ACCEPT     all  --  172.20.0.0/24        172.20.0.0/24       
4    DROP       all  --  192.168.100.0/24     172.20.0.0/24       
5    DROP       all  --  172.20.0.0/24        192.168.100.0/24    
6    LOG        all  --  0.0.0.0/0            0.0.0.0/0            LOG flags 0 level 4 prefix "FIREWALL-DROP: "



In [ ]:
await firewall_tools.remove_firewall_rule([1])

INFO:tools:Removing firewall rules: [1]


{'rules_removed_current_epoch': ['REMOVED rule #1:    ACCEPT     tcp  --  192.168.100.0/24     172.20.0.7']}

In [23]:
honeypot_ip = '172.20.0.2' 
attacker_ip = '192.168.100.2'

await firewall_tools.add_allow_rule(source_ip=attacker_ip, dest_ip=honeypot_ip)
await firewall_tools.add_allow_rule(source_ip=honeypot_ip, dest_ip=attacker_ip)

INFO:tools.firewall_tools:Adding allow rule: ALLOW 192.168.100.2 -> 172.20.0.2 (tcp)
INFO:tools.firewall_tools:Adding allow rule: ALLOW 172.20.0.2 -> 192.168.100.2 (tcp)


{'rules_added_current_epoch': 'ALLOW 172.20.0.2 -> 192.168.100.2 (tcp)'}